In [ ]:
import vtk
from vtk.numpy_interface import dataset_adapter as dsa
from vmtk import vmtkscripts
import numpy as np
from itertools import repeat

In [ ]:
surfacereader = vmtkscripts.vmtkSurfaceReader()
surfacereader.InputFileName = "/Users/rick/projects/vmtk/vmtk-test-data/input/aorta-surface.vtp"
surfacereader.Execute()

In [ ]:
meshgen = vmtkscripts.vmtkMeshGenerator()
meshgen.Surface = surfacereader.Surface
meshgen.Execute()

In [ ]:
writer= vmtkscripts.vmtkMeshWriter()
writer.Mesh = meshgen.Mesh
writer.OutputFileName = "/Users/rick/projects/vmtk/vmtk-test-data/input/aorta-mesh.vtu"
writer.Execute()

In [ ]:
mesh = meshgen.Mesh

In [ ]:
class vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

The unstructured grid dataset consists of arbitrary combinations of any possible cell type. Unstructured grids are defined by points, cells, and cell types. The `CELLS` keyword requires two parameters: the number of cells `n` and the size of the cell list size. The cell list size is the total number of integer values required to represent the list (i. e., sum of numPoints and connectivity indices over each cell). The `CELL_TYPES` keyword requires a single parameter: the number of cells `n`. This value should match the value specified by the `CELLS` keyword. The cell types data is a single integer value per cell that specified cell type (see vtkCell.h)

```
DATASET UNSTRUCTURED_ GRID 
POINTS n dataType 
p 0x p 0y p 0z 
p 1x p 1y p 1z 

... 
p (n-1) x p (n-1) y p (n-1) z 

CELLS n size 
numPoints 0 ,i, j, k, l,... 
numPoints 1 ,i, j, k, l,... 
numPoints 2 ,i, j, k, l,... 
... 
numPoints n-1 ,i, j, k, l,... 
Simple Legacy Formats 5 
CELL_ TYPES n 
type 0 
type 1 
type 2 
... 
type n-1 
```

In [ ]:
ArrayDict = vividict()
returnCellTypesAsStrings = True

In [ ]:
def ConvertFlatCellsArrayToList(cells, cellLocations):
    '''convert a flat array defining cells into a list of numpy arrays which each define a cell
    
    this function is the inverse of vmtknumpytomesh.ConvertListToFlatCellsArray(cellPointIdsList)
    
    arguments: 
        - cells: 1D array of format [npointsCell1, cell1PointId_1, .., cell1PointId_npointsCell1, 
                                     npointsCell2, cell2PointId_1, .., cell2PointId_npointsCell2, 
                                     ... 
                                     npointsCell(nCells), cell(nCells)PointId_1, .. cell(nCells)PointId_npointsCell(nCells)]
                                     
        - cellLocations: flat array of size = nCells. each element in the array defines starts a new cell 
                        (a location of npointCellFoo) in the cells array
    
    returns:
        - cellPointIdsList: list of numpy arrays (in same order defined in cells), where each array contains 
                            the cellPointIds for that specific cell. Note: array lengths are not constant, 
                            each cell can have a different number of constituent points.
    '''

    splitArrays = np.split(cells, cellLocations[1:]) # start first split after end of Cell1, at nPointsCell2
    
    cellPointIdsList = []
    for subArray in splitArrays:
        cellPointIdsList.append(subArray[1:]) # get rid of npointsCell(foo), only keep cellPointIds
    
    return cellPointIdsList

In [ ]:
def ConvertListToFlatCellsArray(cellPointIdsList):
    '''convert a list of numpy arrays defining each cell into a flat array defining cells. 
    
    This function is the inverse of vmtkmeshtonumpy.ConvertFlatCellsArrayToList(cells, cellLocations)
    
    arguments: 
        - cellPointIdsList: list of numpy arrays (in same order defined in cells), where each array contains 
                            the cellPointIds for that specific cell. Note: array lengths are not constant, 
                            each cell can have a different number of constituent points.
    
    returns:
    
        - cells: 1D array of format [npointsCell1, cell1PointId_1, .., cell1PointId_npointsCell1, 
                                     npointsCell2, cell2PointId_1, .., cell2PointId_npointsCell2, 
                                     ... 
                                     npointsCell(nCells), cell(nCells)PointId_1, .. cell(nCells)PointId_npointsCell(nCells)]

        - cellLocations: flat array of size = nCells. each element in the array defines starts a new cell 
                        (a location of npointCellFoo) in the cells array
    '''
    
    cellArrayList = []
    cellLocationsList = [np.array([0])]
    cellIndex = 0
    for cellPointIdArray in cellPointIdsList:
        numPointsInArray = cellPointIdArray.size
        cellArray = np.concatenate((np.array([numPointsInArray]), cellPointIdArray))
        cellArrayList.append(cellArray)
        
        cellIndex += cellArray.size
        cellLocationsList.append(np.array([cellIndex]))
    
    cellLocations = np.concatenate(cellLocationsList[:-1])
    cells = np.concatenate(cellArrayList)
    
    return cells, cellLocations

In [ ]:
wp = dsa.WrapDataObject(mesh)

# (npoints, 3) array of xyz coordinates. values in Cells index to the 
# rows of this array (Cells.max() = Points.shape[0] - 1)
points = np.array(wp.Points)
ArrayDict['Points'] = points

# flat list of locations which index the beginning of a cell in the 
# Cell array (same size as CellTypes and CellEntityIds)
cellLocations = np.array(wp.CellLocations)

# flat array which defines the npoints/cell and indexes of rows 
# in Points array which define each cell's XYZ locations
cells = np.array(wp.Cells)

cellPointIdsList = ConvertFlatCellsArrayToList(cells, cellLocations)
ArrayDict['Cells']['CellPointIds'] = cellPointIdsList


# flat array of shape == cellEntityIds == cellLocations which defines
# the intiger descriptor of the VTK_CELL_TYPE for each cell in cells 
cellTypes = np.array(wp.CellTypes)
ArrayDict['Cells']['CellTypes'] = cellTypes



if returnCellTypesAsStrings == True:
    typeDict = vividict()
    uniqueCellTypes = np.unique(cellTypes)
    for cellType in uniqueCellTypes:
        typeDict[cellType] = vtk.vtkCellTypes.GetClassNameFromTypeId(cellType)
    
    ArrayDict['Cells']['CellTypesAsStrings'] = typeDict
    
for cellDataKey in wp.CellData.keys():
    cellData = wp.CellData.GetArray(cellDataKey)
    ArrayDict['CellData'][cellDataKey] = cellData
    
for pointDataKey in wp.PointData.keys():
    pointData = wp.PointData.GetArray(pointDataKey)
    ArrayDict['PointData'][pointDataKey] = pointData
    

In [ ]:
gridData = vtk.vtkUnstructuredGrid()

In [ ]:
print('converting points')
points = vtk.vtkPoints()
for xyzPoint in ArrayDict['Points']:
    points.InsertNextPoint(xyzPoint)

gridData.SetPoints(points)

In [ ]:
#tests
a = mesh.GetPoints()

a.GetPoint(5)

b = gridData.GetPoints()

b.GetPoint(5)

for i in range(mesh.GetNumberOfPoints()):
    ap = np.array(a.GetPoint(i))
    bp = np.array(b.GetPoint(i))
    if np.allclose(ap, bp) == False:
        print(f'ap: {ap}, bp: {bp}')

In [ ]:
if 'Cells' in ArrayDict.keys():
    print('hi')

In [ ]:
if 'Cells' not in ArrayDict.keys():
    raise KeyError('Cells Key not found in ArrayDict')

In [ ]:
cpIdsList = ArrayDict['Cells']['CellPointIds']
cTypes = ArrayDict['Cells']['CellTypes']
reconCells, reconCellLocations = ConvertListToFlatCellsArray(cpIdsList)

In [ ]:
print(CellTypesArray)

In [ ]:
for idx, cType in enumerate(cTypes):
    idType.InsertNextValue(cType)

In [ ]:
print(idType)

In [ ]:
cellArray = vtk.vtkCellArray()

In [ ]:
len(cpIdsList)

In [ ]:
cellArray.SetNumberOfCells(len(cpIdsList))
cellArray.SetCells()

In [ ]:
cc = mesh.GetCell(3)

In [ ]:
print(cc)

In [ ]:
cellData.SetNumberOfTuples(ArrayDict['Cells']['CellPointIds'])
for 

In [ ]:
if 'PointData' in ArrayDict.keys():
    print('converting point data')
    for pointKey in ArrayDict['PointData'].keys():

        if np.issubdtype(ArrayDict['PointData'][pointKey].dtype, float):
            pointDataArray = vtk.vtkFloatArray()
        else:
            for checkDt in [int, np.uint8, np.uint16, np.uint32, np.uint64]:
                if np.issubdtype(ArrayDict['PointData'][pointKey].dtype, checkDt):
                    pointDataArray = vtk.vtkIntArray()
                    break
                else:
                    continue

        try:
            pointDataComponents = ArrayDict['PointData'][pointKey].shape[1]
        except IndexError:
            pointDataComponents = 1

        if pointDataArray:
            pointDataArray.SetNumberOfComponents(pointDataComponents)
            pointDataArray.SetName(pointKey)

            if pointDataComponents == 1:
                pointDataArray.SetNumberOfValues(ArrayDict['PointData'][pointKey].size)
                for index, pointData in enumerate(ArrayDict['PointData'][pointKey]):
                    pointDataArray.SetValue(index, pointData)
                gridData.GetPointData().SetActiveScalars(pointKey)
                gridData.GetPointData().SetScalars(pointDataArray)
            else:
                for pointData in ArrayDict['PointData'][pointKey]:
                    pointDataArray.InsertNextTuple(pointData)
                gridData.GetPointData().SetActiveVectors(pointKey)
                gridData.GetPointData().SetVectors(pointDataArray)

print('converting cell data')
for cellKey in ArrayDict['CellData'].keys():

    if cellKey == 'CellPointIds':
        cellDataArray = vtk.vtkCellArray()
        cellDataTypes = []
        for cellId in ArrayDict['CellData']['CellPointIds']:
            numberOfCellPoints = cellId.size
            cellDataArray.InsertNextCell(numberOfCellPoints)
            for cellPoint in cellId:
                cellDataArray.InsertCellPoint(cellPoint)
                cellDataTypes.append(int)

        gridData.SetCells(cellDataTypes, cellDataArray)

    else:

        if np.issubdtype(ArrayDict['CellData'][cellKey].dtype, float):
            cellDataArray = vtk.vtkFloatArray()
        if np.issubdtype(ArrayDict['CellData'][cellKey].dtype, int):
            cellDataArray = vtk.vtkIntArray()

        try:
            cellDataComponents = ArrayDict['CellData'][cellKey].shape[1]
        except IndexError:
            cellDataComponents = 1

        cellDataArray.SetNumberOfComponents(cellDataComponents)
        cellDataArray.SetName(cellKey)

        if cellDataComponents == 1:
            cellDataArray.SetNumberOfValues(ArrayDict['CellData'][cellKey].size)
            for index, cellData in enumerate(ArrayDict['CellData'][cellKey]):
                cellDataArray.SetValue(index, cellData)
            gridData.GetCellData().SetActiveScalars(cellKey)
            gridData.GetCellData().SetScalars(cellDataArray)

        else:
            for cellData in self.ArrayDict['CellData'][cellKey]:
                pointDataArray.InsertNextTuple(cellData)
            gridData.GetCellData().SetActiveVectors(cellKey)
            gridData.GetCellData().SetVectors(cellDataArray)


In [ ]:
c = gridData.GetCellData()

In [ ]:
print(c.GetArrayName(0))

In [ ]:
d = mesh.GetCellData()

In [ ]:
d.GetNumberOfArrays()

In [ ]:
viewer = vmtkscripts.vmtkMeshViewer()
viewer.Mesh = meshclipper.Mesh
viewer.Execute()